# Hyperparameter Optimization with Agent
In this notebook we give an example of how to use the agent framework to let the agent do a hyperparameter optimization of the user-provided code.

#### Run the task
Let's take a look at the command we will run at the end of this tutorial to get a high-level idea of what components we need to understand and in what order.
From the `agent/` directory, run the following command in your command line

```bash
$ python -u src/agent/start.py task=hyperopt_interact llm@agent.llm=fschat/deepseek-coder-7b-instruct-v1.5 method=direct-hyperopt
```

First, we see that we need a `task=hyperopt_interact`. This defined through a `configs/task/hyperopt_interact.yaml` file that points to the `HyperOptTask` class.
Then we notice that we have an LLM agent with the backend being [fschat/deepseek-coder-6.7b-instruct](https://huggingface.co/deepseek-ai/deepseek-coder-6.7b-instruct). This is configured in the `configs/llm/fschat/` directory.
Finally, we have to define a `method=direct-hyperopt` through a `configs/method/direct-hyperopt.yaml` file that describes the sequence of actions the agent needs to execute in order to do the task.

This notebook will have the following structure:
- Structure your workspace as expected by the task
- Look at the pre-defined configurations (`task` and `method`)
- Give some information regarding the code, the inner workings of the task
- Additional details about Bayesian Optimization and in particular the HEBO algorithm
- Finally, dive into the example `code.py`

#### Working Example
The example we use in this tutorial is the code for a Kaggle competition, namely the [spaceship-titanic competition](https://www.kaggle.com/competitions/spaceship-titanic).
We give a code example adapted from a public notebook and show how to optimize the model hyperparameters automatically with the `agent` framework.
This is an example of a kaggle competition code adapted from [this notebook](https://www.kaggle.com/code/mikhailnaumov/spaceship-titanic).
This code is saved in `./worspace/code/code.py`.
The data associated with the competition can be downloaded [here](https://www.kaggle.com/competitions/spaceship-titanic/data) and saved into `./workspace/data/train.csv`..

## Structure your Workspace
Start by creating a workspace with the following files:

```
agent/
 | workspace/
 | | hyperopt/
 | | python_path.txt
 | | | code/
 | | | | code.py      
 | | | data/
 | | | | train.csv  
``` 

#### Your python environment
Because the `agent` will run your python script, it needs to be aware of the path to your interpreter.
Make sure to have the path to your python interpreter inside `python_path.txt`.

## Configure your Task and your Flow
First you need the configuration file related to the task. 
This is already pre-defined in `configs/task/hyperopt_interact.yaml`.
It specifies where the task class is.
It also has some default details about the agent which are overwritten in the command line command from above.

```yaml
# @package _global_
agent:
  pre_action_flow: ???
  prompt_builder:
    template_paths:
      - hypefault

task:
  _target_: agent.tasks.hyperoptHyperOpt

  workspace_path: ./workspae/hyperopt

  name: hyperopt
  description:
  subtask: null
  version: v0.1

max_episodes: 1
max_env_steps: 1
```

To run the task, you need to create a `Flow` which is pre-defined `configs/method/direct-hyperopt.yaml`.
That file describes the sequence of commands that the Agent will follow.
In our case, we have a `LoopFlow` of a `DecisionFlow`. 
At each step of the `LoopFlow`, the agent can choose between two commands: `'optimize'` and `'terminate'`. 
- `'optimize'`: command `UseTool` with `HyperOpt` tool: run `HEBO` on the hyperparameters of the model found in `code.py` and overwrite the code with their optimal value
- `'terminate'`: command `Act`: terminate the process

```yaml
# @package _global_
agent:
  pre_action_flow:
    _target_: agent.commands.SequentialFlow
    name: hyperopt_example
    description: Entire HyperOpt pipeline example
    sequence:
      - _target_: agent.commands.LoopFlow
        max_repetitions: 2
        allow_early_break: true
        loop_body:
          _target_: agent.commands.UseTool
          prompt_template: hyperopt.jinja
          tool:
            _target_: agent.tools.hyperopt_tool.HyperOptTool
            bo_steps: 2
            path_to_python: './workspace/hyperopt/python_path.txt'
            workspace_path: ${task.workspace_path}
      - _target_: agent.commands.Act
        name: terminate
        description: save model script with final optimized hyperparameters
  prompt_builder:
    template_paths: [ "hyperopt/" ]
    default_kwargs:
      cot_type: zero_shot
```

## The Task, the Tool and the BO

##### Details about the HyperOptTool
The agent receives the model code from `code.py` and identifies the part that is related to the model. 
It identifies the hyperparameters of that model (i.e. the ones exposed in your code, e.g. as kwargs) and writes a `DesignSpace` object. 
This object is the search space for `HEBO` that defines which parameters of the model to tune and what values they are allowed to take. 
The code in `code.py` is then _wrapped_ in a BO loop in which the model parameters will be changed according to the BO algorithm.
After the specified amount of steps, the best parameters will be written in the code, overwriting the original ones.
The final code will be saved in `workspace/hyperopt/code/best_code.py` while the original code will remain untouched in `workspace/hyperopt/code/code.py`.

##### Details about HEBO
[`HEBO`](https://arxiv.org/abs/2012.03826) is a Bayesian Optimization algorithm that comes with a library that can be simply `pip`-installed, has a simple API and works well for a large variety of black-box optimization tasks.
It requires a `DesignSpace` object to run, which is essentially a dictionary that describes each hyperparameter with its name, type and space of definition.

## Let's Dive into the Code!
The file `code.py` is the code that loads the `train.csv` file, fits the model and prints the score metric.
At the end of the process, this code will be copied and the model parameters will be overwritten with optimal hyperparameters found by the HEBO algorithm.

##### Additional Instructions
Make sure that the *modelling* part of your code, i.e. the part you want to opitmize with `HyperOptTask` is between special markers `# @MODEL_START@` and `# @MODEL_END@` so that the model code can be extracted easily and added to the agent prompt.
Make sure that in your code the last line in a `print` statement, `print("ACCURACY:", accuracy)` if it is a classification task and `print("R2_SCORE:", r2_score)` if it is a regression task.

The full code is given further below but for now let's focus on each part separately

##### Loading the data
Here we load the data, reset the index of the dataframes and split into train and validation

```python
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.impute import KNNImputer

train_data = pd.read_csv("./workspace/hyperopt/data/train.csv", index_col=0)
n = train_data.shape[0]
train, val = train_data.iloc[:int(n * 0.8)], train_data.iloc[int(n * 0.8):]
train.reset_index(drop=True, inplace=True)
val.reset_index(drop=True, inplace=True)
```

##### Feature engineering
Here the features are created, mainly it is a matter of creating Age groups and treating columns related to expenses.
We also need to convert categorical variables into numerical variables for the model.

```python
Expenses_columns = ['RoomService', 'FoodCourt', 'Spa', 'VRDeck', 'ShoppingMall']


def features(data):
    data.loc['Age_group'] = 0
    data.loc[(train['Age'] > 0) & (data['Age'] <= 5), 'Age_group'] = 1
    data.loc[(train['Age'] > 5) & (data['Age'] <= 10), 'Age_group'] = 2
    data.loc[(train['Age'] > 10) & (data['Age'] <= 20), 'Age_group'] = 3
    data.loc[(train['Age'] > 20) & (data['Age'] <= 30), 'Age_group'] = 4
    data.loc[(train['Age'] > 30) & (data['Age'] <= 50), 'Age_group'] = 5
    data.loc[(train['Age'] > 50) & (data['Age'] <= 60), 'Age_group'] = 6
    data.loc[(train['Age'] > 60) & (data['Age'] <= 70), 'Age_group'] = 7
    data.loc[(train['Age'] > 70) & (data['Age'] <= 100), 'Age_group'] = 8
    data.Age_group = data.Age_group.astype(float)

    data['RoomService'] = np.where(data['CryoSleep'] == True, 0, data['RoomService'])
    data['FoodCourt'] = np.where(data['CryoSleep'] == True, 0, data['FoodCourt'])
    data['ShoppingMall'] = np.where(data['CryoSleep'] == True, 0, data['ShoppingMall'])
    data['Spa'] = np.where(data['CryoSleep'] == True, 0, data['Spa'])
    data['VRDeck'] = np.where(data['CryoSleep'] == True, 0, data['VRDeck'])

    data['Group'] = data['PassengerId'].astype(str).str[:4].astype(float)
    data[['Deck', 'Number', 'Side']] = data['Cabin'].str.split('/', expand=True)

    data['Expenses'] = data.loc[:, Expenses_columns].sum(axis=1)
    data.loc[:, ['CryoSleep']] = data.apply(lambda x: True if x.Expenses == 0 and pd.isna(x.CryoSleep) else x, axis=1)
    data['VIP'] = np.where(data['CryoSleep'] == 0, True, False).astype(object)

    data['Name'] = data['Name'].fillna('Unknown Unknown')
    data.loc[:, ['Surname']] = data.Name.str.split(expand=True)[1]

    return data


train = features(train.copy())
val = features(val.copy())
```

##### Drop target column and ID 

```python
train = train.drop(['PassengerId', 'Name', 'Cabin'], axis=1)
val = val.drop(['PassengerId', 'Name', 'Cabin'], axis=1)
X_train = train.drop('Transported', axis=1)
y_train = train.Transported.astype('int')
X_val = val.drop('Transported', axis=1)
y_val = val.Transported.astype('int')
```

##### Fill Missing Values

```python
imputer = KNNImputer(n_neighbors=5)
num_features = X_train.select_dtypes('float64').columns.to_list()
cat_features = X_train.select_dtypes('object').columns.to_list()


def missing(data):
    data[cat_features] = data[cat_features].infer_objects(copy=False).fillna('None').astype('category')
    data[num_features] = imputer.fit_transform(data[num_features])
    return  data


X_train = missing(X_train)
X_val = missing(X_val)
```

##### Fit Model
Please note that here we mark this part of the code with the patters mentionned above.

```python
# @MODEL_START@
cat = CatBoostClassifier(learning_rate=0.01, depth=9, l2_leaf_reg=8, random_state=42, eval_metric='Accuracy', iterations=300, early_stopping_rounds=20)
train_pool = Pool(X_train, y_train, cat_features=cat_features)
cat.fit(train_pool)
# @MODEL_END@
```

##### Make final prediction

```python
val_preds = []
results = []
y_pred_val = cat.predict(X_val)
accuracy = (y_pred_val == np.array(y_val)).sum() / len(y_val)
print("ACCURACY:", accuracy)
```

## Final Remarks
Once you have followed all the above instructions, the task will run automatically.
The LLM Agent will write the `DesignSpace` and run HEBO multiple times until the budget is exhausted.
You can simply read the code in `workspace/hyperopt/code/best_code.py` to check the optimal hyperparameters found.

# Creating all files for you

In [ ]:
import os
import pandas as pd
import numpy as np
# suppose all workspace is present in agent/tutorials/workspace/ then copy over files needed everywhere and remove them at the end
cwd = os.getcwd()
parentwd = os.path.dirname(cwd)
print('current working dir', cwd)
print('parent working dir', parentwd)

In [ ]:
# copy example workspace
os.makedirs('../workspace/hyperopt', exist_ok=True)
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/hyperopt ../workspace/')

In [ ]:
# coopy config files
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/deepseek-coder-7b-instruct-v1.5.yaml {parentwd}/configs/llm/fschat/deepseek-coder-7b-instruct-v1.5.yaml') 
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/hyperopt_interact.yaml {parentwd}/configs/task/hyperopt_interact.yaml') 
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/direct-hyperopt.yaml {parentwd}/configs/method/direct-hyperopt.yaml')


In [ ]:
# copy task and tool files
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/hyperopt.py {parentwd}/src/agent/tasks/hyperopt.py') 
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/hyperopt_tool.py {parentwd}/src/agent/tools/hyperopt_tool.py')
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/hyperopt_utils.py {parentwd}/src/agent/utils/hyperopt_utils.py')

In [ ]:
# copy prompts
os.makedirs('../src/agent/prompts/templates/hyperopt', exist_ok=True)
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/external_action.jinja {parentwd}/src/agent/prompts/templates/hyperopt/external_action.jinja') 
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/flow_loop_continue.jinja {parentwd}/src/agent/prompts/templates/hyperopt/flow_loop_continue.jinja') 
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/hyperopt.jinja {parentwd}/src/agent/prompts/templates/hyperopt/hyperopt.jinja') 
os.system(f'cp -r {cwd}/hyperopt_tutorial_files/system_prompt.jinja {parentwd}/src/agent/prompts/templates/hyperopt/system_prompt.jinja')

In [ ]:
# modiyfing UseTool Command
with open(f'{parentwd}/src/agent/commands/core.py', 'r') as f:
    original_core = f.read()

with open(f'{cwd}/hyperopt_tutorial_files/core.py', 'r') as f:
    modified_core = f.read()

with open(f'{parentwd}/src/agent/commands/core.py', 'w') as f:
    f.write(modified_core)
    
# adding keys to the memory
with open(f'{parentwd}/src/agent/memory.py', 'r') as f:
    original_memory = f.read()

with open(f'{cwd}/hyperopt_tutorial_files/memory.py', 'r') as f:
    modified_memory = f.read()

with open(f'{parentwd}/src/agent/memory.py', 'w') as f:
    f.write(modified_memory)

# Run Command

```bash
python -u src/agent/start.py task=hyperopt_interact llm@agent.llm=fschat/deepseek-coder-7b-instruct-v1.5 method=direct-hyperopt
```

# Removing files for you

In [ ]:
import os
os.remove(f"{parentwd}/configs/method/direct-hyperopt.yaml")
os.remove(f"{parentwd}/configs/task/hyperopt_interact.yaml")
os.remove(f"{parentwd}/configs/llm/fschat/deepseek-coder-7b-instruct-v1.5.yaml")
os.remove(f"{parentwd}/src/agent/tasks/hyperopt.py")
os.remove(f"{parentwd}/src/agent/tools/hyperopt_tool.py")
os.remove(f"{parentwd}/src/agent/utils/hyperopt_utils.py")
os.remove(f"{parentwd}/src/agent/prompts/templates/hyperopt/external_action.jinja")
os.remove(f"{parentwd}/src/agent/prompts/templates/hyperopt/flow_loop_continue.jinja")
os.remove(f"{parentwd}/src/agent/prompts/templates/hyperopt/hyperopt.jinja")
os.remove(f"{parentwd}/src/agent/prompts/templates/hyperopt/system_prompt.jinja")
os.system(f"rm -r {parentwd}/workspace/hyperopt/*")

with open(f'{parentwd}/src/agent/commands/core.py', 'w') as f:
    f.write(original_core)

with open(f'{parentwd}/src/agent/commands/memory.py', 'w') as f:
    f.write(original_memory)